## Build a clean ndpi test set
#### remove background and blurred, using conv filter 

In [1]:
import os
import cv2
import numpy as np

In [2]:
rootpath="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\test_ndpi"
#set the paths to save the 64x64 sutiles according to their labels.
path_64_pos="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\test_ndpi\\testset\\nor_test"
path_64_neg="E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\test_ndpi\\testset\\tum_test"

In [3]:
dirs = []
for root, folders, files in os.walk(rootpath, topdown=False):
    for name in folders:
        if name.startswith('HMNT'):
            dirs.append(root+'\\'+name)
# print(dirs)

In [4]:
kernel = np.array(([-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1]), np.float32)
threshold = 200
threshold_filter = 15

In [7]:
for dir in dirs:
    print(dir+'\\512_image_label')
    
    limg = []
    llabel = []
    for root, folders, files in os.walk(dir+'\\512_image_label', topdown=False):
        for f in files:
            if f.endswith('tif'):
                limg.append(root+'\\'+f)
            elif f.endswith('tiff'):
                llabel.append(root+'\\'+f)
                
    for i in range(len(limg)):
        img = cv2.cvtColor(cv2.imread(limg[i], -1), cv2.COLOR_BGR2RGB)
        label = cv2.imread(llabel[i], -1)/255
        nsub = 0
        img_subtile = np.ndarray(shape=(64,64,64,3), dtype='uint8')
        label_subtile = np.ndarray(shape=(64,64,64)) #already resize to 0-1, if set dtype to uint8, everything will be 0
        
        for c in range(0, 511, 64):
            c_64 = c + 64
            for r in range(0, 511, 64):
                r_64 = r + 64
                img_subtile[nsub,:,:,:] = img[r:r_64,c:c_64,:]
                label_subtile[nsub,:,:] = label[r:r_64,c:c_64]
                nsub = nsub + 1
                
        for j in range(nsub):
            m_sub = np.mean(img_subtile[j,:,:,:])
            y = cv2.filter2D(img_subtile[j,:,:,:], -1, kernel)
            for a in range(y.shape[0]):
                for b in range(y.shape[1]):
                    for c in range(y.shape[2]):
                        if y[a,b,c] < 0:
                            y[a,b,c] = 0
                        if y[a,b,c] > 255:
                            y[a,b,c] = 255
            m_filtered_sub = np.mean(y)
            
            if (m_sub < threshold) & (m_filtered_sub > threshold_filter):
                if sum(sum(label_subtile[j,:,:] == 3/255)) > 2048:
                    nneg = len([x for x in os.listdir(path_64_neg) if x.endswith('.tif')])
                    nneg = str(nneg).zfill(10)
                    cv2.imwrite(path_64_neg + '\\tum_' + nneg + '.tif', cv2.cvtColor(img_subtile[j,:,:,:], cv2.COLOR_RGB2BGR))
                else:
                    npos = len([x for x in os.listdir(path_64_pos) if x.endswith('.tif')])
                    npos = str(npos).zfill(10)
                    cv2.imwrite(path_64_pos + '\\nor_' + npos + '.tif', cv2.cvtColor(img_subtile[j,:,:,:], cv2.COLOR_RGB2BGR))
                    
# print(limg)
# print(llabel)

E:\deeplearning\Hepatocarcinomes\data\5x\test_ndpi\HMNT0001\512_image_label


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\deeplearning\\Hepatocarcinomes\\data\\5x\\test_ndpi\\testset\\nor_test'